In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.opt import TerminationCondition
import time 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from data.data_functions import *
from simulation.simulation_models import *

In [120]:
#################### Optimeringsmodell ####################
def labor_scheduling(df_index:list, demand:list, MaxStaff:int\
                     , patients_staff:int, availability:int, ServiceLevel:float, night: bool, possibleshifts: list, 
                     shiftlengths: list, earliestshiftstart: int, latestshiftstart: int, minemp: list):
    '''
    Function holding the actual model.

    params:
            df_index: Expected value list. The dates for each row in the dataset. 
            demand: Expected value list. The demand of incoming patients each time period.
            MaxStaff. Expected value integer. The maximum staff allowed in the post each shift/time period. 
            patients_staff: Expected value integer. Number of patients each staff can handle simultaneously. 
            availability: Expected value integer. Number of staff ready to be put into a shift. 
            ServiceLevel: Expected value float. How good should the service be? If 1 then zero waiting time for patients before getting attention. 

    output: returns the model. 
    ''' 
    # Creating model instance
    model = pyo.ConcreteModel()

    #### Sets ####
    model.setTime = pyo.Set(initialize=df_index)
    model.setPossibleShifts = pyo.Set(initialize = possibleshifts)
    model.setShiftlengths = pyo.Set(initialize = shiftlengths)


    #### Parameters ####
    
    # Static variable of night vs day
    if night == True:
        need = 0.33
    else:
        need = 1

    # Demand of Patients
    demand_dict = {df_index[i]: demand[i] for i in range(len(df_index))}
    model.D = pyo.Param(model.setTime, initialize = demand_dict, within = pyo.Integers)
    D = model.D

    # Maximum capacity of staff for each shift
    model.MaxStaff = pyo.Param(initialize = MaxStaff, within = pyo.Integers)
    Cap = model.MaxStaff

    # Number of patients each staff can handle at the same time
    model.PatientsPerStaff = pyo.Param(initialize = patients_staff, within = pyo.Integers)
    PPS = model.PatientsPerStaff

    # Available staff at the time of an occuring shift
    availability_dict = {df_index[i]: availability for i in range(len(df_index))}
    model.AvailableStaff = pyo.Param(model.setTime, initialize = availability_dict, within = pyo.Integers)
    A = model.AvailableStaff

    # Service Level
    model.ServiceLevel = pyo.Param(initialize= ServiceLevel, within = pyo.PercentFraction)
    SL = model.ServiceLevel

    # Cost of shift
    model.ShiftCost = pyo.Param(initialize = 10000, within = pyo.Integers)
    C = model.ShiftCost

    # Earliest shift start
    model.earliestshiftstart = pyo.Param(initialize = earliestshiftstart, within = pyo.Integers)
    ESS = model.earliestshiftstart
    
    # Latest shift start
    model.latestshiftstart = pyo.Param(initialize = latestshiftstart, within = pyo.Integers)
    LSS = model.latestshiftstart

    # Minimum Employees at shift
    minemp_dict = {possibleshifts[i]: minemp[i] for i in range(len(possibleshifts))}
    model.min_emp = pyo.Param(model.setPossibleShifts, initialize = minemp_dict, within = pyo.Integers)
    ME = model.min_emp


    #### Decision Variables ####
    
    # Number of staff members allocated to a shift
    model.Staff_Allocated = pyo.Var(model.setTime, within = pyo.NonNegativeIntegers, bounds=(0, None))
    x = model.Staff_Allocated

    # Binary shift variable -> allocated/used shifts
    model.shiftused = pyo.Var(model.setPossibleShifts, model.setTime, model.setShiftlengths, within = pyo.Binary)
    y = model.shiftused

    # Total costs
    model.totcost = pyo.Var(within = pyo.NonNegativeIntegers, bounds = (0,None))
    z = model.totcost


    #### Objective Function ####

    # Minimize total staff allocated across all shifts
    model.obj_min_staff = pyo.Objective(expr=sum(x[s] for s in model.setTime), sense=pyo.minimize)



    #### Constraints ####

    # Demand Coverage Constraint
    model.C1 = pyo.ConstraintList()
    for t in model.setTime:
        model.C1.add(expr = x[t] >= need*(SL*(D[t]/PPS)))

    # Staff Availability
    model.C2 = pyo.ConstraintList()
    for t in model.setTime:
        model.C2.add(expr = x[t] <= A[t])

    # Capacity
    model.C3 = pyo.ConstraintList()
    for t in model.setTime:
        model.C3.add(expr = x[t] <= Cap)


    # Min staffing
    model.C4 = pyo.ConstraintList()
    for s in model.setPossibleShifts:
        for t in model.setTime:
            for l in model.setShiftlengths:
                model.C4.add(expr = x[t] >= y[s, t, l] * ME[s])
    
    # Cost
    model.C5 = pyo.Constraint(rule= z == sum(C * x[t] for t in model.setTime))

    # Shift Start
    # model.C6 = pyo.ConstraintList()
    # for s in model.setPossibleShifts:
    #     for t in model.setTime:
    #         for l in model.setShiftlengths:
    #             if t.hour < ESS or t.hour > LSS:
    #                 model.C6.add(expr=y[s, t, l] == 0)



    # Ensure at least one shift covers each time period
    model.C7 = pyo.ConstraintList()
    for t in model.setTime:
        model.C7.add(expr=sum(y[s, t, l] for s in model.setPossibleShifts for l in model.setShiftlengths) == 1)



    return model


#################### Optimeringsalgoritme ####################
def optimize_staffing(model):
    '''
    Function to run and optimize the model.

    params:
            model: The actual mathematical model to be optimized.  

    output: returns the optimized model, the status, the objective function value, and the staffed allocated each shift/time period. 
    ''' 
    opt = SolverFactory('glpk')
    status = opt.solve(model, tee=True)

    # Check if the model was solved successfully
    if status.solver.termination_condition != TerminationCondition.optimal:
        print("Solver did not converge to an optimal solution.")
        return None, status, None, None

    # Retrieve objective value
    obj = pyo.value(model.obj_min_staff)

    # Retrieve staff allocated for each shift
    staff_allocated = [pyo.value(model.Staff_Allocated[s]) for s in model.setTime]
    total_cost = pyo.value(model.totcost)

    results_list = []
    for s in model.setPossibleShifts:
        for t in model.setTime:
            for l in model.setShiftlengths:
                print(pyo.value(model.shiftused[s, t, l]))
                if pyo.value(model.shiftused[s, t, l]) > 0:
                    results_list.append({'Shift': s, 'Start_Time': t, 'Length': l, 'Allocated': 'Yes'})

    shifts_used = pd.DataFrame(results_list)

    return model, status, obj, staff_allocated, shifts_used, total_cost


#################### Plot av Optimal bemanning ####################
def staff_opt_plot(staff_allocated: list, fin: int, siv: int, unn: int):
    '''
    Function to plot the optimized model.

    params:
            staff_allocated: Expected value list. The list of optimized staff allocated to each shift/time period. 

    output: returns a plot. 
    ''' 
    # Plot staffing requirements
    plt.figure(figsize=(10, 6))
    plt.plot(staff_allocated, marker='o', color='g', linestyle='-')
    plt.axhline(y=fin, color="r", linewidth = 2, linestyle = "-", label = "Fin")
    plt.axhline(y=siv, color="b", linewidth = 2, linestyle = "-", label = "SiV")
    plt.axhline(y=unn, color="y", linewidth = 2, linestyle = "-", label = "UNN")
    plt.title('Bemanning for post hos Finnmarksykehuset')
    plt.xlabel('Time i perioden')
    plt.ylabel('Antall nødvendig bemanning')
    plt.grid(True)
    plt.tight_layout()
    plt.legend(loc = "upper right")
    plt.show()

In [2]:
fin_hf_med_path = "./data/med_pasientstrøm_time_2024.xlsx"
fin_hf_kir_path = "./data/kir_pasientstrøm_time_2024.xlsx"

fin_data_hourly = create_hourly_dataframe(fin_hf_med_path, fin_hf_kir_path)



fin_data_hourly["Belegg"] = fin_data_hourly.apply(calculate_patients, axis=1)
fin_data_hourly["skift_type"] = fin_data_hourly.apply(add_shift_type, axis=1)

fin_data_hourly.head(10)

,År,Måned,Uke,Dag,DatoTid,Timer,post,helg,Antall inn på post,Antall pasienter ut av Post,Belegg,skift_type
0,2024,January,1,Monday,2024-01-01 00:00:00,0,medisinsk,0,2.0,0.0,2.0,natt
1,2024,January,1,Monday,2024-01-01 00:00:00,0,kirurgisk,0,0.0,0.0,2.0,natt
2,2024,January,1,Monday,2024-01-01 01:00:00,1,medisinsk,0,0.0,0.0,2.0,natt
3,2024,January,1,Monday,2024-01-01 01:00:00,1,kirurgisk,0,0.0,0.0,2.0,natt
4,2024,January,1,Monday,2024-01-01 02:00:00,2,kirurgisk,0,0.0,0.0,2.0,natt
5,2024,January,1,Monday,2024-01-01 02:00:00,2,medisinsk,0,0.0,0.0,2.0,natt
6,2024,January,1,Monday,2024-01-01 03:00:00,3,kirurgisk,0,0.0,0.0,2.0,natt
7,2024,January,1,Monday,2024-01-01 03:00:00,3,medisinsk,0,1.0,0.0,3.0,natt
8,2024,January,1,Monday,2024-01-01 04:00:00,4,kirurgisk,0,0.0,0.0,3.0,natt
9,2024,January,1,Monday,2024-01-01 04:00:00,4,medisinsk,0,1.0,0.0,4.0,natt


In [3]:
fin_data_hourly_med = fin_data_hourly[fin_data_hourly["post"]=="medisinsk"]
fin_data_hourly_kir = fin_data_hourly[fin_data_hourly["post"]=="kirurgisk"]

next_year_med = create_forecast_hourly(fin_data_hourly_med, "medisinsk")
next_year_kir = create_forecast_hourly(fin_data_hourly_kir, "kirurgisk")

next_year = pd.concat([next_year_med, next_year_kir], axis=0).sort_values("DatoTid").reset_index()
next_year.drop(["index"], axis=1, inplace=True)
next_year["skift_type"] = next_year.apply(add_shift_type, axis=1)

fin_data_hourly["Prediksjoner pasientstrøm"] = np.nan
fin_data_hourly["Prediksjoner belegg"] = np.nan

fin_data_hourly = pd.concat([fin_data_hourly, next_year], axis=0).sort_values("DatoTid").reset_index()
fin_data_hourly.drop(["index"], axis=1, inplace=True)
fin_data_hourly

,År,Måned,Uke,Dag,DatoTid,Timer,post,helg,Antall inn på post,Antall pasienter ut av Post,Belegg,skift_type,Prediksjoner pasientstrøm,Prediksjoner belegg
0,2024,January,1,Monday,2024-01-01 00:00:00,0,medisinsk,0,2.0,0.0,2.0,natt,NaN,NaN
1,2024,January,1,Monday,2024-01-01 00:00:00,0,kirurgisk,0,0.0,0.0,2.0,natt,NaN,NaN
2,2024,January,1,Monday,2024-01-01 01:00:00,1,medisinsk,0,0.0,0.0,2.0,natt,NaN,NaN
3,2024,January,1,Monday,2024-01-01 01:00:00,1,kirurgisk,0,0.0,0.0,2.0,natt,NaN,NaN
4,2024,January,1,Monday,2024-01-01 02:00:00,2,kirurgisk,0,0.0,0.0,2.0,natt,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27597,2025,October,42,Tuesday,2025-10-14 22:00:00,22,medisinsk,0,NaN,NaN,NaN,kveld,0.0,18.0
27598,2025,October,42,Tuesday,2025-10-14 23:00:00,23,kirurgisk,0,NaN,NaN,NaN,natt,0.0,23.0
27599,2025,October,42,Tuesday,2025-10-14 23:00:00,23,medisinsk,0,NaN,NaN,NaN,natt,0.0,29.0
27600,2025,October,42,Wednesday,2025-10-15 00:00:00,0,medisinsk,0,NaN,NaN,NaN,natt,1.0,1.0


In [4]:
data = fin_data_hourly
post = "medisinsk"
weekend = False
predictions = False
year = 2024
avg_length_of_stay = 3  # Gjennomsnittlig liggetid (dager)
shifts_per_day = 3  # Antall skift per dag
iterations = 1000  # Antall simuleringer
month = ["May", "June", "July", "August", "September", "October"]
shift_type = None
scenario = None
total_beds = 26 # 26 hvis ukedag eller helg med, kir: hverdag = 22, helg = 17
curr_sit = 8
# Med post: hverdag: dag: 12, kveld: 12, natt:4 
# Med post: Helg: 9, 9, 4
# Kir post: Hverdag: 10, 8, 3
# Kir post: Helg: 8, 7, 3

In [132]:
data_opt = opt_dataset(dataset=data, post=post, year=year, shift_type=shift_type, month=month, weekend=weekend, predictions=predictions)
data_opt.head(10)

# Parameters for specific model
df_index = data_opt.DatoTid
demand = data_opt["Antall inn på post"] + round(data_opt["Belegg"])
MaxStaff = 15
PPS = 3
availability = 100
ServiceLevel = 0.8
night = False
possibleshifts = ["dag", "kveld", "natt"]
shiftlengths = [5,6,7,8,9,10]
earliestshiftstart = 6
latestshiftstart = 22 
minemp = [3,3,1]
# costs = 

model = labor_scheduling(df_index, demand, MaxStaff, PPS, availability, ServiceLevel, night, possibleshifts, shiftlengths, earliestshiftstart, latestshiftstart, minemp)
model, status, obj, staff_allocated, shifts_used, total_cost = optimize_staffing(model)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\Consumer\AppData\Local\Temp\tmpkhsn60hx.glpk.raw --wglp
 C:\Users\Consumer\AppData\Local\Temp\tmphjnacn1n.glpk.glp --cpxlp C:\Users\Consumer\AppData\Local\Temp\tmp_tj000wf.pyomo.lp
Reading problem data from 'C:\Users\Consumer\AppData\Local\Temp\tmp_tj000wf.pyomo.lp'...
C:\Users\Consumer\AppData\Local\Temp\tmp_tj000wf.pyomo.lp:414137: warning: lower bound of variable 'x11427' redefined
C:\Users\Consumer\AppData\Local\Temp\tmp_tj000wf.pyomo.lp:414137: warning: upper bound of variable 'x11427' redefined
62833 rows, 54265 columns, 165649 non-zeros
54265 integer variables, 51408 of which are binary
465545 lines were read
Writing problem data to 'C:\Users\Consumer\AppData\Local\Temp\tmphjnacn1n.glpk.glp'...
351295 lines were written
GLPK Integer Optimizer 5.0
62833 rows, 54265 columns, 165649 non-zeros
54265 integer variables, 51408 of which are binary
Preprocessing...
444 constraint coefficient(s) w

In [133]:
shifts_used.head(60)

,Shift,Start_Time,Length,Allocated
0,natt,2024-05-01 00:00:00,10,Yes
1,natt,2024-05-01 01:00:00,10,Yes
2,natt,2024-05-01 02:00:00,10,Yes
3,natt,2024-05-01 03:00:00,10,Yes
4,natt,2024-05-01 04:00:00,10,Yes
5,natt,2024-05-01 05:00:00,10,Yes
6,natt,2024-05-01 06:00:00,10,Yes
7,natt,2024-05-01 07:00:00,10,Yes
8,natt,2024-05-01 08:00:00,10,Yes
9,natt,2024-05-01 09:00:00,10,Yes


In [6]:
data_opt = opt_dataset(dataset=data, post=post, year=year, shift_type=shift_type, month=month, weekend=weekend, predictions=predictions)
data_opt.head(10)

# Parameters for specific model
df_index = data_opt.DatoTid
demand = data_opt["Antall inn på post"] + round(data_opt["Belegg"])
MaxStaff = 15
PPS = 3
availability = 100
ServiceLevel = 0.8

In [36]:
test = fin_data_hourly[(fin_data_hourly["År"]==2024) & (fin_data_hourly["post"]=="medisinsk")]

In [37]:
test["demand"] = test["Antall inn på post"] + round(test["Belegg"])

C:\Users\Consumer\AppData\Local\Temp\ipykernel_13160\2683838832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["demand"] = test["Antall inn på post"] + round(test["Belegg"])


In [38]:
test = test.groupby(["Timer", "skift_type"])["demand"].mean().reset_index()

In [ ]:
test_hour_list = list(test["Timer"])
test_hour_list = [int(element) for element in test["demand"]]

In [54]:
test_demand = [int(element) for element in test["demand"]]
test_demand = list(test_demand)


In [ ]:
import pyomo.environ as pyo
df_index = test_hour_list
numEmp = 
shifts = ["dag", "kveld", "natt"]
shiftLengths = [4,5,6,7,8,9,10,11,12]

# Model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=df_index)
model.Shifts = pyo.Set(initialize=shifts)
model.PossibleStartTimes = pyo.RangeSet(5, 23)
model.PossibleLengths = pyo.Set(initialize=shiftLengths) 

# Parameters

# Demand of Patients
demand_dict = {df_index[i]: demand[i] for i in range(len(df_index))}
model.D = pyo.Param(model.T, initialize = demand_dict, within = pyo.Integers)
D = model.D

# Number of patients each staff can handle at the same time
model.PatientsPerStaff = pyo.Param(initialize = PPS, within = pyo.Integers)
PPS = model.PatientsPerStaff


# Service Level
model.ServiceLevel = pyo.Param(initialize= ServiceLevel, within = pyo.PercentFraction)
SL = model.ServiceLevel


# cost
cost_per_hour = 50  
model.C = pyo.Param(initialize=cost_per_hour)

# Variables
# Number of employees per hour to meet demand
model.x = pyo.Var(model.T, domain=pyo.NonNegativeIntegers)
# Binary variable indicating if a shift starts at a certain time with a certain length
model.y = pyo.Var(model.Shifts, model.PossibleStartTimes, model.PossibleLengths, domain=pyo.Binary)

# Objective: Minimize total cost of staffing
model.obj = pyo.Objective(expr=sum(model.x[t] * model.C for t in model.T), sense=pyo.minimize)

# Demand Coverage Constraint
model.C1 = pyo.ConstraintList()
for t in model.T:
    model.C1.add(expr = model.x[t] >= (SL*(D[t]/PPS)))

# Shift Coverage Constraints
# Ensure that each hour is covered by an active shift
model.C_shift_coverage = pyo.ConstraintList()
for t in model.T:
    # Sum over all shifts that cover hour `t`
    coverage_expr = sum(
        model.y[s, start, length]
        for s in model.Shifts
        for start in model.PossibleStartTimes
        for length in model.PossibleLengths
        if (start <= t < start + length or start + length > 24 and (t < (start + length) % 24))
    )
    # Ensure that each hour `t` has at least one active shift covering it
    model.C_shift_coverage.add(coverage_expr >= 1)

# Solve
solver = pyo.SolverFactory('glpk')
solver.solve(model, tee=True)

# Output results
for s in model.Shifts:
    for start in model.PossibleStartTimes:
        for length in model.PossibleLengths:
            if pyo.value(model.y[s, start, length]) > 0.5:
                print(f"Shift {s} starts at hour {start} with length {length}")
for t in model.T:
    print(f"Employees needed at hour {t}: {pyo.value(model.x[t])}")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\Consumer\AppData\Local\Temp\tmpv28je509.glpk.raw --wglp
 C:\Users\Consumer\AppData\Local\Temp\tmpu15bzsqy.glpk.glp --cpxlp C:\Users\Consumer\AppData\Local\Temp\tmpqpw26jqa.pyomo.lp
Reading problem data from 'C:\Users\Consumer\AppData\Local\Temp\tmpqpw26jqa.pyomo.lp'...
C:\Users\Consumer\AppData\Local\Temp\tmpqpw26jqa.pyomo.lp:2546: warning: lower bound of variable 'x147' redefined
C:\Users\Consumer\AppData\Local\Temp\tmpqpw26jqa.pyomo.lp:2546: warning: upper bound of variable 'x147' redefined
48 rows, 279 columns, 2064 non-zeros
279 integer variables, 255 of which are binary
2801 lines were read
Writing problem data to 'C:\Users\Consumer\AppData\Local\Temp\tmpu15bzsqy.glpk.glp'...
2490 lines were written
GLPK Integer Optimizer 5.0
48 rows, 279 columns, 2064 non-zeros
279 integer variables, 255 of which are binary
Preprocessing...
24 rows, 255 columns, 2040 non-zeros
255 integer variables, all o